In [4]:
# import warnings
# warnings.filterwarnings('ignore')

import os.path

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

sns.set(style="darkgrid")

SOURCE_DATA_PATH = 'source_data'
DERIVED_DATA_PATH = 'derived_data'

##Data Prep

In [29]:
needs = pd.read_csv(os.path.join(DERIVED_DATA_PATH,'C16B_PUF_cleaned_needs.csv'), na_values=' ', index_col='ID')
wellness = pd.read_csv(os.path.join(DERIVED_DATA_PATH,'C16B_PUF_cleaned_wellness.csv'), na_values=' ', index_col='ID')

km_X = needs.join(wellness).dropna() 

In [28]:
print 'data retention post-drop: {:.2%} percent.'.format(float(len(clf_X))/len(needs))
print '# records retention post-drop: {}'.format(len(clf_X))

data retention post-drop: 69.73% percent.
# records retention post-drop: 211646


So that's not bad.  Let's keep going with our cluster analysis by building our model.

##Building Model

In [31]:
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import pairwise_distances

It'll make most sense for us to run the model over several k's to arrive at the optimal k for our data.  We don't get SSE out of the box with sklearn so we'll write our own.

In [41]:
#SSE get_cluster_centers helper function
def get_cluster_centers(X, labels, k_num):
    CC_list = []
    for k in range(k_num):
        #get mean coordinates of each cluster
        CC_list.append(np.mean(X[labels ==k], axis=0))
    return CC_list

#main SSE function
#for each cluster subtract the mean from each data point to get the error
#then get the magnitude of each error, square it, and sum it
def get_SSE(X, labels):
    pass

    

In [32]:
k_range = range(2,10)

In [ ]:
%timeit

SSEs = []
Sil_coefs = []

for k in k_range:
    #instantiate KMeans object
    km = KMeans(n_clusters=k, random_state=1)
    #fit model to training data
    km.fit(km_X)
    labels = km.labels_
    #store SC & SSE scores for each k run
    Sil_coefs.append(metrics.silhouette_score(km_X, labels, metric='euclidean'))
    SSEs.append(get_SSE(km_X, labels))